### Input files: Price-Today.csv

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {'qty':'{:,}','volbuy':'{:,}',
              'price_x':'{:,.2f}','price_y':'{:,.2f}','change':'{:,.2f}','price':'{:,.2f}',
              'chg_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'percent':'{:,.2f}','net':'{:,.2f}','dividend':'{:,.4f}'}

pd.options.display.max_rows = 31
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-08-04 2025-08-03


In [3]:
# If run after actual work day
#today = today - timedelta(days=2)
print(today)

2025-08-04


In [4]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-08-04
yesterday: 2025-08-01


In [5]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-04 20:20:39


### Restart and Run All Cells

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [10]:
sql = """
SELECT name, date, volbuy, price, active, dividend, period, grade 
FROM buy
ORDER BY name"""
stocks = pd.read_sql(sql, const)
stocks['volbuy'] = stocks['volbuy'].astype('int64')
stocks

,name,date,volbuy,price,active,dividend,period,grade
0,3BBIF,2018-05-17,120000,10.10,1,0.00,3,A3
1,AH,2023-06-08,1200,37.00,1,0.78,1,C1
2,AIMIRT,2023-08-17,12500,10.90,1,0.79,2,A1
3,AWC,2023-06-15,9000,4.96,1,0.07,1,A1
4,BCH,2021-09-07,4000,21.70,1,0.40,3,B3
5,CPNREIT,2022-08-16,55000,18.00,1,0.57,2,C1
6,DIF,2020-08-01,40000,13.10,1,0.89,2,C1
7,GVREIT,2022-08-24,75000,7.64,1,0.80,2,C1
8,IVL,2022-08-29,7200,40.00,1,0.70,3,C3
9,JMART,2023-01-20,6800,33.00,1,0.47,1,A2


In [11]:
names = stocks["name"].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'3BBIF', 'AH', 'AIMIRT', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TOA', 'TVO', 'WHAIR', 'WHART'"

In [12]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [13]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [14]:
df_merge  = pd.merge(stocks, prices, on = 'name', how='inner')
df_merge.shape

(27, 17)

### Convert column name to make it valid for df.query

In [16]:
pd.options.display.max_rows = 30
df_merge['percent'] = df_merge['%change']
df_merge['chg_amt'] = df_merge['volbuy'] * df_merge['change']
df_merge['volbuy'] = df_merge['volbuy'].astype('int64')
colo = ['name', 'volbuy', 'price_x', 'price_y', 'change', 'percent', 'chg_amt']
filter = df_merge['percent'] != 0
df_pct = df_merge[filter][colo].sort_values(by='percent', ascending=True).copy()
df_amt = df_merge[filter][colo].sort_values(by='chg_amt', ascending=True).copy()

In [17]:
df_pct.shape

(21, 7)

In [18]:
df_amt.shape

(21, 7)

In [19]:
df_pct.query('percent >= 3.00').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
17,SCC,600,405.00,203.00,6.00,3.05,"3,600.00"


In [20]:
df_pct.query('percent < -3.00').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt


In [21]:
df_amt.query('chg_amt >= 6000').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
0,3BBIF,"120,000",10.10,6.10,0.05,0.83,"6,000.00"


In [22]:
df_amt.query('chg_amt < -6000').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
16,RCL,"27,000",38.75,28.50,-0.75,-2.56,"-20,250.00"


In [23]:
float_formatter = "฿{:,.2f}"

In [24]:
float_formatter.format(df_amt.query('chg_amt >= 0')['chg_amt'].sum())

'฿27,290.00'

In [25]:
float_formatter.format(df_amt.query('chg_amt < 0')['chg_amt'].sum())

'฿-33,845.00'

In [26]:
float_formatter.format(df_amt.chg_amt.sum())

'฿-6,555.00'

In [27]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-04 20:20:40
